In [1]:
%run LanguageModel.py

In [2]:
%run DataLoader.py

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dl = DataLoader('train', ('de', 'en'), max_length = 50, device = device)

In [5]:
val_dl = DataLoader('dev', ('de', 'en'), languageModels = train_dl.languageModels, max_length = 50, device = device)

In [6]:
%run seq2seq.py


In [8]:
lm1 = train_dl.languageModels[train_dl.languages[0]]
lm2 = train_dl.languageModels[train_dl.languages[1]]
model_config = ModelConfig(input_size = lm1.n_tokens, beam_width = 3, hidden_size = 256, output_size = lm2.n_tokens, rnn_type='gru', bidirectional=False, attention = 'global', max_length=52)
#checkpoint = torch.load("./state_dict.tar")
#s2s = seq2seq(model_config=model_config, state_dict = checkpoint, device = device)
s2s = seq2seq(model_config=model_config, state_dict = None, device = device)

In [9]:
def loss_metric(input, output, ground_truth, nll):
    return nll / ground_truth[0].size(0)

def perplexity(input, output, ground_truth, nll):
    nll /= ground_truth[0].size(0)
    return math.exp(nll)

In [10]:
%run ScorePrinter.py

In [13]:
def train_epochs(epochs, print_every=1000):
    n_iters = len(train_dl)
    score_printer = ScorePrinter("Training", [('NLL', loss_metric),('Perplexity', perplexity)])
    
    for epoch in range(1, epochs+1):
        score_printer.startEpoch(epoch)
        idx_permutation = np.random.permutation(len(train_dl))[:5]
        for i, index in enumerate(idx_permutation):
            input_tensor, target_tensor = train_dl.tensorsFromPos(index)

            loss, output_sentence = s2s.train(input_tensor, target_tensor)
            score_printer.update(input_tensor, target_tensor, output_sentence, loss)
            
            if (i + 1) % print_every == 0:
                score_printer.printAvg(print_every)
                validate(print_every)
        
        score_printer.printAvg(len(train_dl))
        val_avg_score = validate(1)
        train_avg_score = score_printer.get_avg_score()
        print(f"Val_avg_score: {val_avg_score}, train_avg_score: {train_avg_score}")
        score_printer.endEpoch(epoch)

        torch.save(s2s.state_dict(),"./state_dict.tar")

In [14]:
import numpy as np
import math

def validate(n = None):
    score_printer = ScorePrinter("Validation", [('NLL', loss_metric),('Perplexity', perplexity)])
    n = n or len(val_dl)
    idx_permutation_val = np.random.permutation(len(val_dl))[:n]
    for j, val_index in enumerate(idx_permutation_val):
        input_tensor_val, target_tensor_val = val_dl.tensorsFromPos(val_index)
        loss, output_sentence = s2s.evaluate(input_tensor_val, target_tensor_val)
        score_printer.update(input_tensor_val, target_tensor_val, output_sentence, loss)
    score_printer.printAvg(showCount = False)
    return score_printer.get_avg_score()

In [15]:
train_epochs(1, print_every=2)



Epoch 1 started
 
[Training] 2 examples /  NLL: 10.971821401311065 Perplexity: 58304.62929838156  
[Validation]   NLL: 2.348610464132057 Perplexity: 21.94154433962473 Self.count : 2
 
[Training] 4 examples /  NLL: 10.57322051965695 Perplexity: 43433.5407271596  
[Validation]   NLL: 2.207434816180535 Perplexity: 12.668734784347937 Self.count : 2
 
[Training] 5 examples /  NLL: 10.404419718134415 Perplexity: 38107.10927787384  
[Validation]   NLL: 2.009615916126179 Perplexity: 7.460451357906272 Self.count : 1
Self.count : 5
Val_avg_score: {'NLL': 2.009615916126179, 'Perplexity': 7.460451357906272}, train_avg_score: {'NLL': 10.404419718134415, 'Perplexity': 38107.10927787384}


Epoch 1 ended
